In [ ]:
import torch
import torch.nn as nn
import torchquantum as tq
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

# Set up data transformations with augmentation for small dataset
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # resize to a common size
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),  # augment with horizontal flip
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # normalize the images
])

# Load custom dataset for cat and dog classification
dataset = torchvision.datasets.ImageFolder(root='./data_images', transform=transform)

# Split into train and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Define data loaders
batch_size = 4
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define a custom quantum layer with parameters
class QuantumLayer(nn.Module):
    def __init__(self, input_size, output_size):
        super(QuantumLayer, self).__init__()
        # Placeholder; adjust as needed with actual quantum operations
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, x):
        return self.fc(x)

# Define the quantum neural network model
class QuantumNeuralNetwork(nn.Module):
    def __init__(self):
        super(QuantumNeuralNetwork, self).__init__()
        self.q_layer1 = QuantumLayer(64 * 64 * 3, 128)  # Define custom quantum layer
        self.q_layer2 = QuantumLayer(128, 2)  # Output layer for binary classification (cat/dog)

    def forward(self, x):
        x = x.view(-1, 64 * 64 * 3)  # Flatten the input
        x = self.q_layer1(x)
        x = torch.relu(x)
        x = self.q_layer2(x)
        return x

# Initialize the model, loss function, and optimizer
model = QuantumNeuralNetwork()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.5)

# Training loop
for epoch in range(100):  # Train for 10 epochs
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print(f'Epoch {epoch+1}, Batch {i+1}, Loss: {loss.item()}')

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(y_true, y_pred, classes):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(7, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

# Assuming you have y_true and y_pred
y_true = []
y_pred = []

# Evaluate model and collect predictions
model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# Plot confusion matrix for cat and dog classifier
plot_confusion_matrix(y_true, y_pred, classes=["Cat", "Dog"])


In [ ]:
from PIL import Image


# Define transformation for input image to match training
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # resize to match input size
    transforms.ToTensor(),        # convert to tensor
])

# Function to classify an image as a cat or dog
def classify_image(model, image_path):
    # Load the image
    image = Image.open(image_path).convert('RGB')
    
    # Apply transformations
    image = transform(image)
    image = image.unsqueeze(0)  # add batch dimension
    
    # Set model to evaluation mode
    model.eval()
    with torch.no_grad():
        # Get model prediction
        output = model(image)
        _, predicted = torch.max(output, 1)
        
    # Map prediction to label
    class_names = ["Cat", "Dog"]
    prediction = class_names[predicted.item()]
    
    return prediction


image_path = input("Enter the path of the image: ")
prediction = classify_image(model, image_path)
print(f'The model predicts this image is a: {prediction}')


In [ ]:
image_path = input("Enter the path of the image: ")
prediction = classify_image(model, image_path)
print(f'The model predicts this image is a: {prediction}')